# Forcasting  

In this notebook we will predict the closing prices for the next month

In [1]:
# Import Libraries
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [2]:
# Load DataSet
data= pd.read_csv('./data/engineered_weekly_stock_market.csv')

data.head()

,companyName,date,openingPrice,highestPrice,lowestPrice,closingPrice,volume,year,month,day_of_month,week_of_year,price_range,price_change,weekly_return,log_volume,moving_avg_4,ema_4,volatility_4
0,AMEN BANK,2014-06-16,-0.104873,-0.105238,-0.105602,-0.105532,-0.111736,2014,6,16,25,-0.069502,-0.067207,-0.608059,-0.506038,-0.134304,-0.105530,-0.129781
1,AMEN BANK,2014-06-23,-0.105491,-0.105851,-0.106085,-0.106023,-0.105668,2014,6,23,26,-0.077417,-0.056514,-0.504264,0.336594,-0.134304,-0.105727,-0.129781
2,AMEN BANK,2014-06-30,-0.105983,-0.106064,-0.106466,-0.106136,-0.099024,2014,6,30,27,-0.068063,-0.024434,-0.159442,0.583623,-0.134304,-0.105891,-0.129781
3,AMEN BANK,2014-07-07,-0.105983,-0.105814,-0.106708,-0.106036,-0.111046,2014,7,7,28,-0.039998,-0.015879,-0.065430,-0.243828,-0.105453,-0.105948,-0.106254
4,AMEN BANK,2014-07-14,-0.105870,-0.105826,-0.105818,-0.106023,-0.110070,2014,7,14,29,-0.091090,-0.024434,-0.159025,-0.046500,-0.105576,-0.105978,-0.125022
